In [1]:
from __future__ import print_function
import pickle 
import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
from sub import subMNIST       # testing the subclass of MNIST dataset

Files already downloaded
3000
750


# Split Data

In [3]:
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))
                             ])

In [4]:
trainset_original = datasets.MNIST('../data', train=True, download=True,
                                  transform=transform)

Files already downloaded


In [5]:
train_label_index = []
valid_label_index = []
for i in range(10):
    train_label_list = trainset_original.train_labels.numpy()
    label_index = np.where(train_label_list == i)[0]
    label_subindex = list(label_index[:300])
    valid_subindex = list(label_index[300: 1000 + 300])
    train_label_index += label_subindex
    valid_label_index += valid_subindex

## Train Set

In [6]:
trainset_np = trainset_original.train_data.numpy()
trainset_label_np = trainset_original.train_labels.numpy()
train_data_sub = torch.from_numpy(trainset_np[train_label_index])
train_labels_sub = torch.from_numpy(trainset_label_np[train_label_index])

In [7]:
trainset_new = subMNIST(root='./data', train=True, download=True, transform=transform, k=3000)
trainset_new.train_data = train_data_sub.clone()
trainset_new.train_labels = train_labels_sub.clone()

Files already downloaded


In [8]:
trainset_new.train_data.size()

torch.Size([3000, 28, 28])

In [9]:
pickle.dump(trainset_new, open("train_labeled.p", "wb" ))

## Validation Set

In [10]:
validset_np = trainset_original.train_data.numpy()
validset_label_np = trainset_original.train_labels.numpy()
valid_data_sub = torch.from_numpy(validset_np[valid_label_index])
valid_labels_sub = torch.from_numpy(validset_label_np[valid_label_index])

In [11]:
validset = subMNIST(root='./data', train=False, download=True, transform=transform, k=10000)
validset.test_data = valid_data_sub.clone()
validset.test_labels = valid_labels_sub.clone()

Files already downloaded


In [12]:
validset.test_data.size()

torch.Size([10000, 28, 28])

In [13]:
pickle.dump(validset, open("validation.p", "wb" ))

## Unlabeled Data

In [14]:
train_unlabel_index = []
for i in range(60000):
    if i in train_label_index or i in valid_label_index:
        pass
    else:
        train_unlabel_index.append(i)

In [15]:
trainset_np = trainset_original.train_data.numpy()
trainset_label_np = trainset_original.train_labels.numpy()
train_data_sub_unl = torch.from_numpy(trainset_np[train_unlabel_index])
#train_labels_sub_unl = torch.from_numpy(trainset_label_np[train_unlabel_index])
temp = np.empty(47000)
temp.fill(-1)
train_labels_sub_unl = torch.from_numpy(temp)

In [16]:
trainset_new_unl = subMNIST(root='./data', train=True, download=True, transform=transform, k=47000)
trainset_new_unl.train_data = train_data_sub_unl.clone()
trainset_new_unl.train_labels = train_labels_sub_unl.clone()      # Unlabeled!!

Files already downloaded


In [17]:
trainset_new_unl.train_data.size()[0]

47000L

In [18]:
trainset_new_unl.train_labels.size()

torch.Size([47000])

In [19]:
pickle.dump(trainset_new_unl, open("train_unlabeled.p", "wb" ))

# Training

In [20]:
## load from pickle
trainset_import = pickle.load(open("train_labeled.p", "rb"))
validset_import = pickle.load(open("validation.p", "rb"))
unlabelset_import = pickle.load(open("train_unlabeled.p", "rb"))
testset_import = pickle.load(open("test.p", "rb"))

IOError: [Errno 2] No such file or directory: 'test.p'

In [ ]:
train_loader = torch.utils.data.DataLoader(trainset_import, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=64, shuffle=True)
unlabel_loader = torch.utils.data.DataLoader(unlabelset_import, batch_size=32, shuffle=True)
#train_loader = torch.utils.data.DataLoader(trainset_import, batch_size=1, shuffle=True) 
#valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=1, shuffle=True)
#unlabel_loader = torch.utils.data.DataLoader(unlabelset_import, batch_size=1, shuffle=True)

In [ ]:
for label, unlabel in zip(enumerate(train_loader), enumerate(unlabel_loader)):
    batch_idx, (data, target) = label
    batch_idx, (data_un, target_un) = unlabel
    data = torch.cat((data, data_un), 0)
    target = torch.cat((target, torch.LongTensor(32).fill_(-1)), 0)
    #train_loader = batch_idx, (data, target)

In [ ]:
c=0
for label in enumerate(train_loader):
    batch_idx, (data, target) = label
    if c==0:
        print(data)
    c += 1    

In [ ]:
## defining the CNN
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        # Deconvolution
        self.fc3 = nn.Linear(10, 50)
        self.fc4 = nn.Linear(50, 320)
        self.deconv1 = nn.ConvTranspose2d(20, 10, kernel_size=5)
        self.deconv2 = nn.ConvTranspose2d(10, 1, kernel_size=5)
        #Loss
        self.loss_fn = nn.MSELoss()
        

    def forward(self, x):
        x, indices1 = F.max_pool2d(self.conv1(x), 2, return_indices=True) 
        x_CONVI = F.relu(x)
        x, indices2 = F.max_pool2d(self.conv2_drop(self.conv2(x_CONVI)), 2, return_indices=True)
        x_CONVII = F.relu(x)
        x = x_CONVII.view(-1, 20*4*4)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x_NLL = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x_NLL))
        x_DECONVII = F.relu(self.fc4(x))
        x = x_DECONVII.view(self.get_size(x_DECONVII),20,4,4)
        x_DECONVI = F.relu(self.deconv1(F.max_unpool2d(x, indices2 , 2, 2)))
        output = F.relu(self.deconv2(F.max_unpool2d(x_DECONVI, indices1 , 2, 2)))
        return output, F.log_softmax(x_NLL)
    def get_size(self, x):
        return x.size()[0]

## initialize
model = Net()

In [ ]:
## SGD optimization
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
## define two functions to train and test.
# CPU only training
def train(epoch):
    model.train()
    crit = nn.MSELoss() # mean squared error as the loss function
    for label, unlabel in zip(enumerate(train_loader), enumerate(unlabel_loader)):
        batch_idx, (data, target) = label
        n = data.size()[0]
        batch_idx, (data_un, target_un) = unlabel
        data = torch.cat((data, data_un), 0)
        target = torch.cat((target, torch.LongTensor(32).fill_(-1)), 0)
        #Convert to variables
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        out_decode, output = model(data)
        loss_delta = crit(out_decode, data)
        loss = F.nll_loss(output[0:n], target[0:n]) + loss_delta
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            ## every 10 batchs print a status update
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), 2*len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

## now test
def test(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        _, output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [ ]:
## testing the previously defined functions?
for epoch in range(1):
    train(epoch)
    test(epoch, valid_loader)

In [ ]:
##TEMP Files##
conv1 = nn.Conv2d(1, 10, kernel_size=5)
conv2 = nn.Conv2d(10, 20, kernel_size=5)
conv2_drop = nn.Dropout2d()
fc1 = nn.Linear(320, 50)
fc2 = nn.Linear(50, 10)
# Deconvolution
fc3 = nn.Linear(10, 50)
fc4 = nn.Linear(50, 320)
deconv1 = nn.ConvTranspose2d(20, 10, kernel_size=5)
deconv2 = nn.ConvTranspose2d(10, 1, kernel_size=5)
mseLoss = nn.MSELoss()
count = 0
for batch_idx, (data, target) in enumerate(train_loader):
        if count > 0:
            break   
        data, target = Variable(data), Variable(target)
        print ('Data1: ', data.size())
        x, indices1 = F.max_pool2d(conv1(data), 2, return_indices=True) 
        x_CONVI = F.relu(x)
        x, indices2 = F.max_pool2d(conv2_drop(conv2(x_CONVI)), 2, return_indices=True)
        x_CONVII = F.relu(x)
        x = x_CONVII.view(-1, 320)
        x = F.relu(fc1(x))
        #x = F.dropout(x, training=training)
        x_NLL = F.relu(fc2(x))
        x = F.relu(fc3(x_NLL))
        x_DECONVII = F.relu(fc4(x))
        x = x_DECONVII.view(1,20,4,4)
        x_DECONVI = F.relu(deconv1(F.max_unpool2d(x, indices2 , 2, 2)))
        output = F.relu(deconv2(F.max_unpool2d(x_DECONVI, indices1 , 2, 2)))
        print(mseLoss(x_CONVI, x_DECONVI))
        count = count + 1

In [ ]:
params = list(model.parameters())
for s in range(len(params)):
    print(params[s].size())

# Create Sample Submission

In [ ]:
testset = datasets.MNIST('../data', train=False, transform=transform)

In [ ]:
pickle.dump(testset, open("test.p", "wb" ))

In [ ]:
test_loader = torch.utils.data.DataLoader(testset,batch_size=64, shuffle=False)

## Test Accuuracy

In [ ]:
test(1, test_loader)

In [ ]:
label_predict = np.array([])
model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    temp = output.data.max(1)[1].numpy().reshape(-1)
    label_predict = np.concatenate((label_predict, temp))

In [ ]:
label_predict

In [ ]:
label_true = test_loader.dataset.test_labels.numpy()

In [ ]:
diff_array = label_true - label_predict

In [ ]:
len(np.where(diff_array != 0)[0])

In [ ]:
import pandas as pd
true_label = pd.DataFrame(label_true, columns=['label'])
true_label.reset_index(inplace=True)
true_label.rename(columns={'index': 'ID'}, inplace=True)

In [ ]:
true_label.head()

In [ ]:
predict_label = pd.DataFrame(label_predict, columns=['label'], dtype=int)
predict_label.reset_index(inplace=True)
predict_label.rename(columns={'index': 'ID'}, inplace=True)

In [ ]:
predict_label.head()

In [ ]:
predict_label.to_csv('sample_submission.csv', index=False)
true_label.to_csv('true_label.csv', index=False)